# EnergyPlus Utility

## Installation From Github Repo and Setup Environment

In [12]:
!pip install -q "energy-plus-utility @ git+https://github.com/janithcyapa/energy-plus-utility.git@main"

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install --force-reinstall -q "energy-plus-utility @ git+https://github.com/janithcyapa/energy-plus-utility.git@main"

  Cloning https://github.com/janithcyapa/energy-plus-utility.git (to revision main) to /tmp/pip-install-ag26ke_r/energy-plus-utility_b6721b887fa24e6bba3e12ce6ad43caf
  Running command git clone --filter=blob:none --quiet https://github.com/janithcyapa/energy-plus-utility.git /tmp/pip-install-ag26ke_r/energy-plus-utility_b6721b887fa24e6bba3e12ce6ad43caf
  Resolved https://github.com/janithcyapa/energy-plus-utility.git to commit a968e81c74eec3305828f98119fa52375f193adb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.1 MB/s eta 0:00:00
  Using cached simdkalman-1.0.2-py2.py3-none-any.whl.metadata (1.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 8.4 MB/s eta 0:00:00
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
Using cached sim

`prepare_colab_eplus` - Download and Install EnergyPlus 25-1 in `/root/EnergyPlus-22-1-0` 

In [22]:
import importlib.metadata
ver = importlib.metadata.version("energy-plus-utility")
print(f"\n✅ Installed 'energy-plus-utility' version: {ver}")



✅ Installed 'energy-plus-utility' version: 0.2.1+4


In [4]:
from eplus import prepare_colab_eplus
prepare_colab_eplus()

# Usage

In [23]:
from eplus.core import EPlusUtil

### Setup Basics

In [24]:
# In your main script or core.py
# verbose, 0 - no logging, 1- verbose logging, 2 - debug logging
OUT_DIR = "/simulation/eplus_out"
sim = EPlusUtil(verbose=2, out_dir=OUT_DIR)

# Reset state before setting model
sim.reset_state()
# Delete previous output directory
sim.delete_out_dir()
# Clear previous outputs
sim.clear_eplus_outputs(patterns="eplusout.*")

Initialized new EnergyPlus state.
EnergyPlus state has been reset.
Deleted output directory: /simulation/eplus_out


### Set Model using local files

In [16]:
# Specify the EnergyPlus Import
EPLUS_DIR = "/root/EnergyPlus-25-1-0"

# Define the Simulation Model
IDF = f"{EPLUS_DIR}/ExampleFiles/5ZoneAirCooled.idf"
# Select Weather Data
EPW = f"{EPLUS_DIR}/WeatherData/USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw"
# Set the Model for Simulation
sim.set_model(IDF, EPW)

EnergyPlus state has been reset.
Model set: IDF='/root/EnergyPlus-25-1-0/ExampleFiles/5ZoneAirCooled.idf', EPW='/root/EnergyPlus-25-1-0/WeatherData/USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw', OUT_DIR='/simulation/eplus_out'


### Set Model From URL

In [25]:
url_idf = "https://raw.githubusercontent.com/janithcyapa/DHCA-Framework/refs/heads/main/System%20Models/MultiZoneOffice/MultizoneOffice.idf"
url_epw = "https://raw.githubusercontent.com/janithcyapa/DHCA-Framework/refs/heads/main/System%20Models/MultiZoneOffice/LKA_Colombo-Katunayake.434500_SWERA.epw"

# Set the Model for Simulation
sim.set_model_from_url(url_idf, url_epw)

EnergyPlus state has been reset.
Model set: IDF='/simulation/eplus_out/MultizoneOffice.idf', EPW='/simulation/eplus_out/LKA_Colombo-Katunayake.434500_SWERA.epw', OUT_DIR='/simulation/eplus_out'


### Run Basic Simualtion

In [26]:
sim.run_dry_run(include_ems_edd=False,reset=True,design_day=False)

AttributeError: 'EPlusUtil' object has no attribute 'run_dry_run'